In [1]:
#Importing Libraries


import numpy as np 
import pandas as pd 
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import nltk
nltk.download('stopwords')
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
/kaggle/input/fake-and-real-news-dataset/True.csv
/kaggle/input/fake-and-real-news-dataset/Fake.csv


In [2]:
print(stopwords.words('english')) #these words dont have any value on our dataset

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# **Reading Files**

In [3]:
df_true = pd.read_csv('/kaggle/input/fake-and-real-news-dataset/True.csv')

In [4]:
df_fake = pd.read_csv('/kaggle/input/fake-and-real-news-dataset/Fake.csv')

# **Data Exploration & Pre-Proccessing**

In [5]:
df_true.shape

(21417, 4)

In [6]:
df_fake.shape

(23481, 4)

In [7]:
df_true.isnull().sum()

title      0
text       0
subject    0
date       0
dtype: int64

In [8]:
df_fake.isnull().sum()

title      0
text       0
subject    0
date       0
dtype: int64

Now , lets adding our 2 df in a one df

In [9]:
df_true['label'] = 0
df_fake['label'] = 1
df = pd.concat([df_true, df_fake], ignore_index=True)
df


,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0
...,...,...,...,...,...
44893,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",1
44894,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",1
44895,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",1
44896,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",1


In [10]:
df.dtypes

title      object
text       object
subject    object
date       object
label       int64
dtype: object

In [11]:
# merging the subject name and news title
df['content'] = df['subject']+' '+df['title']

In [12]:
df.head()

,title,text,subject,date,label,content
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0,"politicsNews As U.S. budget fight looms, Repub..."
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0,politicsNews U.S. military to accept transgend...
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0,politicsNews Senior U.S. Republican senator: '...
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0,politicsNews FBI Russia probe helped by Austra...
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0,politicsNews Trump wants Postal Service to cha...


# **Stemming**

we use stemming to return a word to its root like :
running --< run
making --< make ...etc
and this procedure makes the work with text easier.

In [13]:
port_stem = PorterStemmer()

In [14]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [15]:
# applying our function on content column
df['content'] = df['content'].apply(stemming)

In [16]:
print(df['content'])

0        politicsnew u budget fight loom republican fli...
1        politicsnew u militari accept transgend recrui...
2        politicsnew senior u republican senat let mr m...
3        politicsnew fbi russia probe help australian d...
4        politicsnew trump want postal servic charg muc...
                               ...                        
44893    middl east mcpain john mccain furiou iran trea...
44894    middl east justic yahoo settl e mail privaci c...
44895    middl east sunnistan us alli safe zone plan ta...
44896    middl east blow million al jazeera america fin...
44897    middl east u navi sailor held iranian militari...
Name: content, Length: 44898, dtype: object


In [17]:
# applying our function on text column (it takes many time,so you can apply it on content only )

#df['text'] = df['text'].apply(stemming)


To simplfy, i will use content column only and i will ignore (text) because takes many time to process , but its better to use (text) if you have time . 

# **Creating Model**

In [18]:
#difining x,y
X = df['content'].values
Y = df['label'].values

In [19]:
#vectorizing x 
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [20]:
#spliting x,y
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.8,random_state=0,shuffle=True)

In [21]:
#applying Algorithm
LR = LogisticRegression()
LR.fit(x_train, y_train)

LogisticRegression()

# **Model Evaluation**

In [22]:
pred = LR.predict(x_test)
accuracy = accuracy_score(pred, y_test)


In [23]:
print('The accuracy score is  : ', accuracy)

The accuracy score is  :  0.9998607979064005
